In [1]:
from InitialGraphs import Graphdata
from InitialPattern import Pattern
from GraphPattern import PatternMarge
import time

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = len(dict)*n//100

    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

fileName = 'NCI1'
pers = 5 # N-hop Pattern Filtering Ratio
psize = 10000 # 𝐹𝑟𝑒𝑞𝑢𝑒𝑛𝑡 𝑃𝑎𝑡𝑡𝑒𝑟𝑛 𝑀𝑎𝑛𝑎𝑔𝑚𝑒𝑛𝑡 Table Size
batchsize = 3
thlist1 = []
thlist2 = []
FPMT = {}
FPpattern = {}
ExceptionList = {}
iterations = 0
FpPt = {}

for i in range(11):
    s1 = time.time()
    IGPath = './Gen_Evolution_set/' + fileName + '/' + fileName + '_' + str(i) + '.edges'
    lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels'

    Separator = ','

    DataT0 = Graphdata(IGPath, lbPath, Separator)
    
    Th1 = Top_nPercent(DataT0.OhP, pers)
    Th2 = Top_nPercent(DataT0.OhP, pers+1)
    
    DxMtrix = [0 for col in range(batchsize+3)]    
    iterations = i%batchsize
    
    if iterations < batchsize:
        DataP0 = Pattern(DataT0.OhP, ExceptionList, pers, psize)
        if i > batchsize:
            for e in ExceptionList:
                FPMT[e][iterations] = 0
                
        FP = DataP0.NhP
        print(len(FP))
        thlist1.append(Th1)
        thlist2.append(Th2)
        thavg1 = sum(thlist1)/batchsize
        thavg2 = sum(thlist2)/batchsize
        
        for ohp in FP.keys():
            ohpRe = ohp.split('/')
            ohpRe.reverse()
            ohpReverse = '/'.join(ohpRe)
            
            if ohp not in FPMT or ohpReverse not in FPMT:
                DxMtrix = [0 for col in range(batchsize+3)]
                DxMtrix[0] = ohp
                DxMtrix[iterations+1] = FP[ohp]
                FPMT[ohp] = DxMtrix
            else:
                if ohpReverse in FPMT:
                    FPMT[ohpReverse][iterations+1] = FP[ohp]
                else:
                    FPMT[ohp][iterations+1] = FP[ohp]
            
            FPMT[ohp][batchsize+1] = sum(FPMT[ohp][1:batchsize+1])/batchsize
            
            if FPMT[ohp][batchsize+1] > thavg1:
                FPMT[ohp][batchsize+2] = 1
            else:
                if FPMT[ohp][batchsize+1] < thavg2:
                    FPMT[ohp][batchsize+2] = -1
                else:
                    FPMT[ohp][batchsize+2] = 0
                    
    for fp in FPMT.keys():
        FpPt[fp] = FPMT[ohp][batchsize+1]
        if FPMT[fp][batchsize+2] == 1:
            FPpattern[fp] = FPMT[fp][batchsize+1]
            ExceptionList[fp] = 1
    
    PM = PatternMarge(DataT0.IG, FpPt, DataT0.lbDict, DataT0.nList)
    Ge = PM.SG
    
    e1 = time.time() - s1
    print(i, e1)

    f = open('./Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers) + '.edges', 'w')

    for e in Ge:
        str_e = e[0] + ',' + e[1] + '\n'
        f.write(str_e)
            
    f.close()

    P_id_list = list(sorted(PM.pidDict.items(), key = lambda item: item[1], reverse = True))
    Gp = P_id_list
    f = open('./Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers) + '.pattern', 'w')

    for e in Gp:
        str_e = e[1] + ',' + e[0] + '\n'
        f.write(str_e)
            
    f.close()

    Gl = list(sorted(PM.SGnl.items()))
    f = open('./Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers) + '.node_labels', 'w')

    for e in Gl:
        str_e = e[0] + ',' + e[1] + '\n'
        f.write(str_e)
            
    f.close()

707
0 78.03214263916016
711


KeyboardInterrupt: 

In [2]:
print(DataP0.Threshold)
print(len(DataP0.NhP))
print(len(PM.SGnl))
print(len(PM.SG))

def convert_size(size_bytes):
    import math
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

import os
file_name = './Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers)
extension = ['.edges', '.node_labels', '.pattern']
file_size = 0
for e in extension:
    file_size = file_size + os.path.getsize(file_name + e) 
    print(e.split('.')[1], convert_size(os.path.getsize(file_name + e)))
    
print('Sum File Size:', convert_size(file_size), 'bytes')

input_file = [IGPath, lbPath]
file_size1 = 0
for ip in input_file:
    file_size1 = file_size1 + os.path.getsize(ip)

3335
711
32148
29894


FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: './Result/NCI1_711_5.edges'